In [25]:
from docx import Document
import re

def reformat_telecom_plans(docx_path, output_path):
    try:
        # 1. docx 파일에서 텍스트 추출
        doc = Document(docx_path)
        full_text = []
        for para in doc.paragraphs:
            if para.text.strip():
                full_text.append(para.text)
        
        content = "\n".join(full_text)

        # 2. 태그 제거
        clean_text = re.sub(r'\\s*', '', content)
        
        # 3. 요금제명 형식 수정
        lines = [line.strip() for line in clean_text.split('\n') if line.strip()]
        formatted_lines = []
        
        for line in lines:
            if line.startswith("요금제명"):
                if formatted_lines:
                    formatted_lines.append("") # 요금제 사이 간격
                formatted_lines.append(f"# {line}")
            else:
                # 항목별 불릿 포인트 추가 (선택 사항, 원치 않으시면 제거 가능)
                formatted_lines.append(f"* {line}")
        
        # 4. 결과 저장
        result = "\n".join(formatted_lines)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(result)
            
        print(f"변환 성공! 파일이 저장되었습니다: {output_path}")

    except Exception as e:
        print(f"오류 발생: {e}")

# --- 실행 부분 ---
# 입력하신 경로를 변수에 담아 실행합니다. (r 접두사를 붙여 경로 오류 방지)
input_path = r'C:/Users/미소정보기술/OneDrive - 인하대학교/바탕 화면/RAG파일/시나리오2번용RAG/기타테스트버전/lgu+_test_v2.docx'
output_path = r'C:/Users/미소정보기술/OneDrive - 인하대학교/바탕 화면/RAG파일/시나리오2번용RAG/기타테스트버전/lgu+.md'


In [26]:
reformat_telecom_plans(input_path, output_path)

변환 성공! 파일이 저장되었습니다: C:/Users/미소정보기술/OneDrive - 인하대학교/바탕 화면/RAG파일/시나리오2번용RAG/기타테스트버전/lgu+.md


## MD문서만들기

In [22]:
import os
import re
from docx import Document

# --- 경로 설정 ---
SOURCE_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\카드_은행_0102"
MD_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\MD_개별파일"

# 처리 대상 리스트
card_list = [
    "삼성카드", "KB국민카드", "롯데카드", "신한카드", "NH농협카드", "하나카드",
    "IBK기업은행", "우리카드", "BC바로카드", "현대카드", "iM뱅크", "케이뱅크",
    "BNK부산은행", "광주은행", "제주은행", "토스뱅크", "우체국", "미래에셋증권"
]

def finalize_unit(unit_dict, card_company):
    """
    수집된 정보를 정형화된 MD 텍스트 포맷으로 변환
    '카드사' 항목을 추가하여 출처를 명확히 함
    """
    ott_combined = ", ".join(unit_dict["OTT_혜택"]) if unit_dict["OTT_혜택"] else "정보 없음"
    annual_fee = unit_dict["연회비"] if unit_dict["연회비"] else "정보 없음"
    target = unit_dict["추천_대상"] if unit_dict["추천_대상"] else "정보 없음"
    
    # 요청하신 대로 '카드사' 라인을 추가함
    return (
        f"# {unit_dict['name']}\n"
        f"카드사 : {card_company}\n"
        f"연회비 : {annual_fee}\n"
        f"OTT 혜택 : {ott_combined}\n"
        f"추천 대상 : {target}"
    )

def generate_md_with_company():
    # 출력 폴더 생성
    os.makedirs(MD_OUTPUT_DIR, exist_ok=True)

    print("📂 카드사 정보가 포함된 MD 문서 생성 시작...")

    for card_name in card_list:
        source_path = os.path.join(SOURCE_DIR, f"{card_name}.docx")
        
        if not os.path.exists(source_path):
            print(f"⚠️ 파일을 찾을 수 없습니다 (건너뜀): {card_name}.docx")
            continue

        doc = Document(source_path)
        raw_lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
        
        products_md = []
        current_unit = None

        for line in raw_lines:
            # 1. 불필요한 태그 제거
            line = re.sub(r'\\s*', '', line)
            
            # 2. 상품명 감지 (스플릿 지점)
            header_match = re.search(r'\[(.*?)\]\s*(.*)', line)
            if header_match:
                if current_unit:
                    # finalize_unit 호출 시 card_name(카드사)을 인자로 전달
                    products_md.append(finalize_unit(current_unit, card_name))
                
                p_name = header_match.group(2).strip()
                current_unit = {
                    "name": p_name,
                    "연회비": "",
                    "OTT_혜택": [],
                    "추천_대상": "",
                    "state": "normal"
                }
            
            elif current_unit:
                # 3. 데이터 매칭 로직
                if "연회비 :" in line:
                    current_unit["연회비"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif "OTT 혜택" in line:
                    current_unit["state"] = "ott"
                elif "추천 대상 :" in line:
                    current_unit["추천_대상"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif current_unit["state"] == "ott":
                    clean_benefit = re.sub(r'^[ln·-]\s+', '', line)
                    current_unit["OTT_혜택"].append(clean_benefit)

        # 마지막 상품 추가
        if current_unit:
            products_md.append(finalize_unit(current_unit, card_name))

        # 4. 파일 저장
        if products_md:
            md_file_path = os.path.join(MD_OUTPUT_DIR, f"{card_name}.md")
            full_md_text = "\n\n---\n\n".join(products_md)
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(full_md_text)
            
            print(f"✅ [{card_name}] MD 생성 완료 (카드사 정보 포함)")

if __name__ == "__main__":
    generate_md_with_company()

📂 카드사 정보가 포함된 MD 문서 생성 시작...
✅ [삼성카드] MD 생성 완료 (카드사 정보 포함)
✅ [KB국민카드] MD 생성 완료 (카드사 정보 포함)
✅ [롯데카드] MD 생성 완료 (카드사 정보 포함)
✅ [신한카드] MD 생성 완료 (카드사 정보 포함)
✅ [NH농협카드] MD 생성 완료 (카드사 정보 포함)
✅ [하나카드] MD 생성 완료 (카드사 정보 포함)
✅ [IBK기업은행] MD 생성 완료 (카드사 정보 포함)
✅ [우리카드] MD 생성 완료 (카드사 정보 포함)
✅ [BC바로카드] MD 생성 완료 (카드사 정보 포함)
✅ [현대카드] MD 생성 완료 (카드사 정보 포함)
✅ [iM뱅크] MD 생성 완료 (카드사 정보 포함)
✅ [케이뱅크] MD 생성 완료 (카드사 정보 포함)
✅ [BNK부산은행] MD 생성 완료 (카드사 정보 포함)
✅ [광주은행] MD 생성 완료 (카드사 정보 포함)
✅ [제주은행] MD 생성 완료 (카드사 정보 포함)
✅ [토스뱅크] MD 생성 완료 (카드사 정보 포함)
✅ [우체국] MD 생성 완료 (카드사 정보 포함)
✅ [미래에셋증권] MD 생성 완료 (카드사 정보 포함)


## 임베딩 및 json 파일 만들기

In [27]:
import os
import re
import json
from sentence_transformers import SentenceTransformer

# --- 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\MD_개별파일"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\개별파일"

# 처리 대상 리스트
card_list = [
    "삼성카드", "KB국민카드", "롯데카드", "신한카드", "NH농협카드", "하나카드",
    "IBK기업은행", "우리카드", "BC바로카드", "현대카드", "iM뱅크", "케이뱅크",
    "BNK부산은행", "광주은행", "제주은행", "토스뱅크", "우체국", "미래에셋증권"
]

# 청크 최대 글자 수
MAX_CHARS_PER_CHUNK = 800

print("🧠 MD 기반 가변 청킹 프로세스 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def run_flexible_chunking():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for card_name in card_list:
        md_path = os.path.join(MD_DIR, f"{card_name}.md")
        
        if not os.path.exists(md_path):
            print(f"⚠️ MD 파일을 찾을 수 없습니다: {card_name}.md")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 1. '---' 구분선으로 개별 상품 유닛 분리 
        units = [u.strip() for u in content.split('---') if u.strip()]
        
        final_chunks = []
        temp_chunk_text = ""
        
        # 2. 글자 수(1000자)에 맞춘 병합 로직
        for unit in units:
            # 현재 누적 텍스트 + 다음 상품 유닛이 1000자를 넘으면 현재까지를 청크로 확정
            if len(temp_chunk_text) + len(unit) > MAX_CHARS_PER_CHUNK:
                if temp_chunk_text:
                    final_chunks.append(temp_chunk_text.strip())
                temp_chunk_text = unit
            else:
                # 1000자 미만이면 계속 병합
                if temp_chunk_text:
                    temp_chunk_text += "\n\n---\n\n" + unit
                else:
                    temp_chunk_text = unit
        
        # 마지막 남은 텍스트 추가
        if temp_chunk_text:
            final_chunks.append(temp_chunk_text.strip())

        # 3. 청크별 임베딩 및 JSON 생성
        card_json_results = []
        for i, chunk_text in enumerate(final_chunks):
            # 첫 번째 상품명을 추출하여 source 필드 대표값으로 사용 
            header_match = re.search(r"#\s*(.+)", chunk_text)
            source_name = header_match.group(1).strip() if header_match else card_name

            dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
            
            card_json_results.append({
                "pk": f"{card_name}_CHUNK_{i+1:02d}",
                "source": card_name,
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": f"{card_name}.md",
                "page": 1,
                "row": 1
            })

        # 4. 카드사별 JSON 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{card_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(card_json_results, f, ensure_ascii=False, indent=4)
        
        print(f"✅ [{card_name}] 청킹 완료: {len(final_chunks)}개의 청크 생성")

if __name__ == "__main__":
    run_flexible_chunking()

🧠 MD 기반 가변 청킹 프로세스 시작...
✅ [삼성카드] 청킹 완료: 8개의 청크 생성
✅ [KB국민카드] 청킹 완료: 8개의 청크 생성
✅ [롯데카드] 청킹 완료: 5개의 청크 생성
✅ [신한카드] 청킹 완료: 12개의 청크 생성
✅ [NH농협카드] 청킹 완료: 4개의 청크 생성
✅ [하나카드] 청킹 완료: 3개의 청크 생성
✅ [IBK기업은행] 청킹 완료: 1개의 청크 생성
✅ [우리카드] 청킹 완료: 3개의 청크 생성
✅ [BC바로카드] 청킹 완료: 2개의 청크 생성
✅ [현대카드] 청킹 완료: 2개의 청크 생성
✅ [iM뱅크] 청킹 완료: 1개의 청크 생성
✅ [케이뱅크] 청킹 완료: 1개의 청크 생성
✅ [BNK부산은행] 청킹 완료: 1개의 청크 생성
✅ [광주은행] 청킹 완료: 1개의 청크 생성
✅ [제주은행] 청킹 완료: 1개의 청크 생성
✅ [토스뱅크] 청킹 완료: 1개의 청크 생성
✅ [우체국] 청킹 완료: 1개의 청크 생성
✅ [미래에셋증권] 청킹 완료: 1개의 청크 생성


## 글자수 900자로 청킹 (최적)

In [28]:
import os
import re
import json
from sentence_transformers import SentenceTransformer

# --- 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\MD_개별파일"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\카드_은행파일"

# 처리 대상 리스트
card_list = [
    "삼성카드", "KB국민카드", "롯데카드", "신한카드", "NH농협카드", "하나카드",
    "IBK기업은행", "우리카드", "BC바로카드", "현대카드", "iM뱅크", "케이뱅크",
    "BNK부산은행", "광주은행", "제주은행", "토스뱅크", "우체국", "미래에셋증권"
]

# 청크 최대 글자 수
MAX_CHARS_PER_CHUNK = 900

print("🧠 MD 기반 가변 청킹 프로세스 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def run_flexible_chunking():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for card_name in card_list:
        md_path = os.path.join(MD_DIR, f"{card_name}.md")
        
        if not os.path.exists(md_path):
            print(f"⚠️ MD 파일을 찾을 수 없습니다: {card_name}.md")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 1. '---' 구분선으로 개별 상품 유닛 분리 
        units = [u.strip() for u in content.split('---') if u.strip()]
        
        final_chunks = []
        temp_chunk_text = ""
        
        # 2. 글자 수(1000자)에 맞춘 병합 로직
        for unit in units:
            # 현재 누적 텍스트 + 다음 상품 유닛이 1000자를 넘으면 현재까지를 청크로 확정
            if len(temp_chunk_text) + len(unit) > MAX_CHARS_PER_CHUNK:
                if temp_chunk_text:
                    final_chunks.append(temp_chunk_text.strip())
                temp_chunk_text = unit
            else:
                # 1000자 미만이면 계속 병합
                if temp_chunk_text:
                    temp_chunk_text += "\n\n---\n\n" + unit
                else:
                    temp_chunk_text = unit
        
        # 마지막 남은 텍스트 추가
        if temp_chunk_text:
            final_chunks.append(temp_chunk_text.strip())

        # 3. 청크별 임베딩 및 JSON 생성
        card_json_results = []
        for i, chunk_text in enumerate(final_chunks):
            # 첫 번째 상품명을 추출하여 source 필드 대표값으로 사용 
            header_match = re.search(r"#\s*(.+)", chunk_text)
            source_name = header_match.group(1).strip() if header_match else card_name

            dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
            
            card_json_results.append({
                "pk": f"{card_name}_CHUNK_{i+1:02d}",
                "source": card_name,
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": f"{card_name}.md",
                "page": 1,
                "row": 1
            })

        # 4. 카드사별 JSON 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{card_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(card_json_results, f, ensure_ascii=False, indent=4)
        
        print(f"✅ [{card_name}] 청킹 완료: {len(final_chunks)}개의 청크 생성")

if __name__ == "__main__":
    run_flexible_chunking()

🧠 MD 기반 가변 청킹 프로세스 시작...
✅ [삼성카드] 청킹 완료: 6개의 청크 생성
✅ [KB국민카드] 청킹 완료: 7개의 청크 생성
✅ [롯데카드] 청킹 완료: 4개의 청크 생성
✅ [신한카드] 청킹 완료: 11개의 청크 생성
✅ [NH농협카드] 청킹 완료: 4개의 청크 생성
✅ [하나카드] 청킹 완료: 3개의 청크 생성
✅ [IBK기업은행] 청킹 완료: 1개의 청크 생성
✅ [우리카드] 청킹 완료: 3개의 청크 생성
✅ [BC바로카드] 청킹 완료: 2개의 청크 생성
✅ [현대카드] 청킹 완료: 1개의 청크 생성
✅ [iM뱅크] 청킹 완료: 1개의 청크 생성
✅ [케이뱅크] 청킹 완료: 1개의 청크 생성
✅ [BNK부산은행] 청킹 완료: 1개의 청크 생성
✅ [광주은행] 청킹 완료: 1개의 청크 생성
✅ [제주은행] 청킹 완료: 1개의 청크 생성
✅ [토스뱅크] 청킹 완료: 1개의 청크 생성
✅ [우체국] 청킹 완료: 1개의 청크 생성
✅ [미래에셋증권] 청킹 완료: 1개의 청크 생성


# 구독상품

In [ ]:
import os
import re
from docx import Document

# --- 경로 설정 ---
# 구독 문서가 있는 경로로 수정
SOURCE_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\시나리오3번용RAG\구독_0102"
MD_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\구독상품\md"

# 구독 서비스 리스트 (폴더 내 파일명에 맞춰 추가)
subscription_list = [
    "KT구독", "T우주패스", "네이버멤버십", "배민클럽", "유독", "티빙"
]

def finalize_unit(unit_dict, service_category):
    ott_combined = ", ".join(unit_dict["OTT_혜택"]) if unit_dict["OTT_혜택"] else "정보 없음"
    price_info = unit_dict["이용료"] if unit_dict["이용료"] else "정보 없음"
    target = unit_dict["추천_대상"] if unit_dict["추천_대상"] else "정보 없음"
    
    return (
        f"# {unit_dict['name']}\n"
        f"서비스 : {service_category}\n"
        f"이용료 : {price_info}\n"
        f"OTT 혜택 : {ott_combined}\n"
        f"추천 대상 : {target}"
    )

def generate_subscription_md():
    os.makedirs(MD_OUTPUT_DIR, exist_ok=True)
    print("📂 구독 상품 정보가 포함된 MD 문서 생성 시작...")

    for service_name in subscription_list:
        source_path = os.path.join(SOURCE_DIR, f"{service_name}.docx")
        
        if not os.path.exists(source_path):
            print(f"⚠️ 파일을 찾을 수 없습니다: {service_name}.docx")
            continue

        doc = Document(source_path)
        raw_lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
        
        products_md = []
        current_unit = None

        for line in raw_lines:
            # 1. 태그 및 불필요한 공백 제거
            line = re.sub(r'\\s*', '', line)
            
            # 2. 상품명 감지 ([KT 구독] 상품명 패턴)
            header_match = re.search(r'\[(.*?)\]\s*(.*)', line)
            if header_match:
                if current_unit:
                    products_md.append(finalize_unit(current_unit, service_name))
                
                p_name = header_match.group(2).strip()
                current_unit = {
                    "name": p_name,
                    "이용료": "",
                    "OTT_혜택": [],
                    "추천_대상": "",
                    "state": "normal"
                }
            
            elif current_unit:
                # 3. 데이터 매칭 (구독 전용 키워드: 이용료, OTT 혜택, 추천 대상)
                if "이용료 :" in line:
                    current_unit["이용료"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif "OTT 혜택" in line:
                    current_unit["state"] = "ott"
                elif "추천 대상 :" in line:
                    current_unit["추천_대상"] = line.split(":", 1)[-1].strip()
                    current_unit["state"] = "normal"
                elif current_unit["state"] == "ott":
                    # 상세 혜택 수집 (요금제 정보 등)
                    clean_benefit = re.sub(r'^[ln·-]\s+', '', line)
                    current_unit["OTT_혜택"].append(clean_benefit)

        if current_unit:
            products_md.append(finalize_unit(current_unit, service_name))

        if products_md:
            md_file_path = os.path.join(MD_OUTPUT_DIR, f"{service_name}.md")
            full_md_text = "\n\n---\n\n".join(products_md)
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(full_md_text)
            
            print(f"✅ [{service_name}] MD 생성 완료")

if __name__ == "__main__":
    generate_subscription_md()

📂 구독 상품 정보가 포함된 MD 문서 생성 시작...
✅ [KT구독] MD 생성 완료
✅ [T우주패스] MD 생성 완료
✅ [네이버멤버십] MD 생성 완료
✅ [배민클럽] MD 생성 완료
✅ [유독] MD 생성 완료
✅ [티빙] MD 생성 완료


In [1]:
import os
import re
import json
import time
from sentence_transformers import SentenceTransformer

# --- 경로 및 설정 ---
MD_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG파일\최종RAG\구독상품\md"
JSON_OUTPUT_DIR = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\구독상품"

# 처리 대상 리스트 (변수명 통일)
subscription_list = [
    "KT구독", "T우주패스", "네이버멤버십", "배민클럽", "유독", "티빙"
]

# 청크 최대 글자 수
MAX_CHARS_PER_CHUNK = 1200

print("🧠 구독 상품 MD 기반 가변 청킹 및 임베딩을 시작합니다...")
# 임베딩 모델 로드
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def run_flexible_chunking():
    os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

    for service_name in subscription_list:
        md_path = os.path.join(MD_DIR, f"{service_name}.md")
        
        if not os.path.exists(md_path):
            print(f"⚠️ MD 파일을 찾을 수 없습니다: {md_path}")
            continue

        with open(md_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 1. '---' 구분선으로 개별 상품 유닛 분리 
        units = [u.strip() for u in content.split('---') if u.strip()]
        
        final_chunks = []
        temp_chunk_text = ""
        
        # 2. 가변 청킹 로직 (900자 기준 병합)
        for unit in units:
            # 다음 유닛을 합쳤을 때 900자를 넘으면 현재까지를 하나의 청크로 생성
            if len(temp_chunk_text) + len(unit) > MAX_CHARS_PER_CHUNK:
                if temp_chunk_text:
                    final_chunks.append(temp_chunk_text.strip())
                temp_chunk_text = unit
            else:
                # 900자 미만이면 '---' 구분선과 함께 병합
                if temp_chunk_text:
                    temp_chunk_text += "\n\n----- \n\n" + unit
                else:
                    temp_chunk_text = unit
        
        # 마지막 남은 텍스트 처리
        if temp_chunk_text:
            final_chunks.append(temp_chunk_text.strip())

        # 3. 청크별 임베딩 생성 및 JSON 구성
        service_results = []
        for i, chunk_text in enumerate(final_chunks):
            # --- gRPC Rate Limit(0.1) 대응: 요청 간 1초 대기 ---
            time.sleep(1.0)
            
            try:
                # 임베딩 생성
                dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
                
                service_results.append({
                    "pk": f"{service_name}_CHUNK_{i+1:02d}",
                    "source": service_name,
                    "text": chunk_text,
                    "dense": dense_vec.tolist(),
                    "file_hash": f"{service_name}.md",
                    "page": 1,
                    "row": 1
                })
            except Exception as e:
                print(f"❌ {service_name} 청크 {i+1} 임베딩 중 오류 발생: {e}")
                time.sleep(10) # 오류 발생 시 대기 시간 연장
                continue

        # 4. 서비스별 개별 JSON 저장
        json_file_path = os.path.join(JSON_OUTPUT_DIR, f"{service_name}.json")
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(service_results, f, ensure_ascii=False, indent=4)
        
        print(f"✅ [{service_name}] 완료: {len(final_chunks)}개의 청크가 {json_file_path}에 저장되었습니다.")

if __name__ == "__main__":
    run_flexible_chunking()
    print("\n🚀 모든 구독 상품의 청킹 및 임베딩 작업이 완료되었습니다.")

KeyboardInterrupt: 